In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import r2_score
from scipy.optimize import curve_fit
from scipy.optimize import leastsq
import scipy.optimize as opt
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
import os
from data_preprocessing import FilteringCurves, ShowResponseCurves
from fitting_curves import FittingColumn, ShowResponseCurvesWithFitting, compute_r2_score
_FOLDER = "./data/"

In [3]:
np.log(10)

2.302585092994046

In [4]:
def TrainTestKernelByDrugLog(train_data, test_data, drug_ids, number_coefficients, kernel='linear', 
                          kernel_dict=None, kernel_param = None,
                          alpha=1, gamma=None, degree=3, coef0=1):
    
    conc_columns= ["fd_num_"+str(i) for i in range(10)]
    response_norm = ['norm_cells_'+str(i) for i in range(10)]
    param_cols = ["param_"+str(i) for i in range(1,5)]
    df_results = test_data[['COSMIC_ID', 'DRUG_ID'] + conc_columns + response_norm + param_cols].copy()
    
    #check whether each coefficient needs its own parameters

    for drug_id in drug_ids:
        indexes_train = train_data[train_data["DRUG_ID"]==drug_id].index
        indexes_test = test_data[test_data["DRUG_ID"]==drug_id].index
    
        X_train = train_data.loc[indexes_train, train_data.columns[26:-4]].values
        X_test = test_data.loc[indexes_test, test_data.columns[26:-4]].values
    
        for i in range(number_coefficients):
            #check whether each coefficient needs its own parameters
            if kernel_dict:
                kernel = kernel_dict[i+1]
                alpha_value = np.float32(kernel_param[kernel]["alpha"][i+1])
                try:
                    gamma_value = np.float32(kernel_param[kernel]["gamma"][i+1])
                    degree_value = np.float32(kernel_param[kernel]["degree"][i+1])
                    coef0_value = np.float32(kernel_param[kernel]["coef0"][i+1])
                except:
                    gamma_value = gamma
                    degree_value = degree
                    coef0_value = coef0
            #if kernel is a string (i.e the same for all parameters)
            else:
                if type(alpha)==dict:
                    alpha_value = alpha[i+1]
                else:
                    alpha_value = alpha
                
                if type(gamma)==dict:
                    gamma_value = gamma[i+1]
                else:
                    gamma_value = gamma
            
                if type(degree)==dict:
                    degree_value = degree[i+1]
                else:
                    degree_value = degree
                
                if type(coef0)==dict:
                    coef0_value = coef0[i+1]
                else:
                    coef0_value = coef0
                    
            #log transformation to limit predicted values
            
            y_train_log = np.log(abs(train_data.loc[indexes_train, "param_"+str(i+1)].values))
            
            kernel_model_log = KernelRidge(kernel=kernel, alpha=alpha_value,
                                            gamma=gamma_value,
                                            degree=degree_value,
                                            coef0=coef0_value)
            
            kernel_model_log.fit(X_train, y_train_log)
            # predictions - log(y), need to transform them back
            y_pred = kernel_model_log.predict(X_test)
            if i!=2:
                df_results.loc[indexes_test, "log_pred_param_"+str(i+1)] = np.exp(y_pred)
            else:
                df_results.loc[indexes_test, "log_pred_param_"+str(3)] = np.exp(-y_pred)
            # usual training for comparison
            y_train = train_data.loc[indexes_train, "param_"+str(i+1)].values

            kernel_model = KernelRidge(kernel = kernel, alpha = alpha_value, gamma=gamma_value, 
                                 degree=degree_value, coef0=coef0_value)
            
            kernel_model.fit(X_train, y_train)
            y_pred = kernel_model.predict(X_test)
            df_results.loc[indexes_test, "pred_param_"+str(i+1)] = y_pred
        
    return df_results

def ComputeErrorByDrug(df_results, pred_param_cols, number_coefficients, metrics="mse", col_name_start =""):
    df_errors_test = pd.DataFrame()
    for drug_id in df_results["DRUG_ID"].unique():
        for i in range(number_coefficients):
            drug_index = df_results[df_results["DRUG_ID"]==drug_id].index
            y_test = df_results.loc[drug_index, "param_"+str(i+1)]
            y_pred = df_results.loc[drug_index, pred_param_cols]
            if metrics == "mse":
                error = mean_squared_error(y_test, y_pred)
            elif metrics == "mae":
                error = mean_absolute_error(y_test, y_pred)
            else:
                print("ERROR: Unknown metrics")
            df_errors_test.loc[drug_id, col_name_start+metrics+"_coef_"+str(i+1)] = error
    return df_errors_test

In [5]:
os.listdir("./results/")

['merged_drug_profiles_sigmoid4_23.csv',
 'filtered_drug_profiles_13.csv',
 'filtered_drug_profiles_12.csv',
 'filtered_drug_profiles_23.csv',
 'filtered_drug_profiles.csv',
 'merged_drug_profiles_sigmoid4_123.csv',
 'filtered_drug_profiles_123.csv']

In [6]:
# NEED TO CHANGE - what file to read
df = pd.read_csv("./results/merged_drug_profiles_sigmoid4_123.csv")
df.shape

(2612, 1100)

In [7]:
conc_columns= ["fd_num_"+str(i) for i in range(10)]
response_norm = ['norm_cells_'+str(i) for i in range(10)]

In [8]:
cell_features = pd.read_csv(_FOLDER+"Cell_Line_Features_PANCAN_simple_MOBEM.tsv", sep="\t")

stat_data = df.groupby(["DRUG_ID"])[["COSMIC_ID"]].count().rename(columns={"COSMIC_ID": "count_cell_lines"})\
            .sort_values("count_cell_lines", ascending=False)
    

drug_features = pd.read_csv(_FOLDER+'/Drug_Features.csv').rename(columns = {"Drug ID": "DRUG_ID"})
statistics = pd.merge(left = stat_data, right = drug_features, how= "left", on = "DRUG_ID").sort_values("count_cell_lines", ascending =False)
statistics.head(10)

,DRUG_ID,count_cell_lines,Drug Name,Synonyms,Target,Target Pathway
0,328,115,SNX-2112,SNX 2112,HSP90,Protein stability and degradation
1,272,107,AR-42,"HDAC-42, AR 42, AR42",HDAC1,Chromatin histone acetylation
2,273,106,CUDC-101,CUDC 101,"HDAC1-10, EGFR, ERBB2",Other
3,170,102,Shikonin,Anchusin,not defined,Other
4,274,100,Belinostat,"PXD101, PXD-101",HDAC1,Chromatin histone acetylation
5,276,100,CAY10603,-,"HDAC1, HDAC6",Chromatin histone acetylation
6,200,87,Dacinostat,"NVP-LAQ824, LAQ824",HDAC1,Chromatin histone acetylation
7,219,86,AT-7519,AT7519,"CDK1, CDK2, CDK4, CDK6, CDK9",Cell cycle
8,180,72,Thapsigargin,Octanoic acid,SERCA,Other
9,346,57,THZ-2-102-1,-,CDK7,Cell cycle


### Forming train and test sets with specified amount of each drug

In [9]:
%%time

# select subsets for each drug and divide each of them into train and test data
# concatenate all the train and test subsets

gr = df.groupby(["DRUG_ID"])["COSMIC_ID"].count()
good_drug_ids = gr[gr>50].index
len(good_drug_ids)

train = pd.DataFrame(columns=df.columns)
test = pd.DataFrame(columns=df.columns)

for drug_id in good_drug_ids:
    df_i = df[df["DRUG_ID"]==drug_id]
    np.random.seed(123)
    indexes = np.random.permutation(df_i.index)
    train_size = int(df_i.shape[0]*0.7)
    indexes_train = indexes[:train_size]
    indexes_test= indexes[train_size:]
    
    train_set = df_i.loc[indexes_train, :]
    test_set = df_i.loc[indexes_test, :]
    
    train = pd.concat([train, train_set], axis=0)
    test = pd.concat([test, test_set], axis=0)
    
print(train.shape, test.shape)

(683, 1100) (300, 1100)
CPU times: user 735 ms, sys: 87.9 ms, total: 823 ms
Wall time: 1.13 s


### Training and testing with only one kernel for all fitting coefficients

In [10]:
short_test = TrainTestKernelByDrugLog(train, test, good_drug_ids, number_coefficients=4, kernel='linear', alpha=1, gamma=None, degree=3, coef0=1)

# parameters: x0, L, k, b
# y = 1 / (L + np.exp(-k*(x-x0)))+b
# Wang 1.0 / (1.0 + np.exp((x-p)/s)   x - dosage [0, 1], p - position, s - shape parameter
short_test[['COSMIC_ID', 'DRUG_ID', "param_1","pred_param_1","param_2", "pred_param_2", 
            "param_3", "pred_param_3", "param_4", "pred_param_4"]].head()

fitting_cols = ["param_"+str(i) for i in range(1,5)]
pred_fitting_cols = ["pred_param_"+str(i) for i in range(1,5)]
log_pred_fitting_cols = ["log_pred_param_"+str(i) for i in range(1,5)]
fitting_function = "sigmoid_4_param"

short_test["r2_fitted"] = compute_r2_score(short_test, x_columns = conc_columns, y_columns = response_norm, 
                              fitting_parameters=fitting_cols, fitting_function = fitting_function)

short_test["r2_predicted"] = compute_r2_score(short_test, x_columns = conc_columns, y_columns = response_norm, 
                              fitting_parameters=pred_fitting_cols, fitting_function = fitting_function)

short_test["r2_predicted_log"] = compute_r2_score(short_test, x_columns = conc_columns, y_columns = response_norm, 
                              fitting_parameters=log_pred_fitting_cols, fitting_function = fitting_function)

# only one case when predicted values was a better fit to original data than directly fitted
print("Predicted values give higher R2 than fitted:")
display(short_test[short_test["r2_predicted"]>short_test["r2_fitted"]][["DRUG_ID","r2_fitted", "r2_predicted"]])

print("Predicted values that give R2 >0.9:", short_test[short_test["r2_predicted"]>0.9][["DRUG_ID","r2_fitted", "r2_predicted"]].shape[0])
display(short_test[short_test["r2_predicted"]>0.9][["DRUG_ID","r2_fitted", "r2_predicted", "r2_predicted_log"]].head())

print("Log Predicted values that give R2 >0.9:", short_test[short_test["r2_predicted_log"]>0.9].shape[0])
display(short_test[short_test["r2_predicted_log"]>0.9][["DRUG_ID","r2_fitted", "r2_predicted", "r2_predicted_log"]].head())

print("Results of direct fitting that gives R2 >0.9:", short_test[short_test["r2_fitted"]>0.9].shape[0])

Predicted values give higher R2 than fitted:


,DRUG_ID,r2_fitted,r2_predicted
1606,219,5.050434e-08,0.71805


Predicted values that give R2 >0.9: 22


,DRUG_ID,r2_fitted,r2_predicted,r2_predicted_log
2419,170,0.998540,0.923729,-0.233380
500,219,0.997481,0.935583,-0.211019
1855,219,0.998097,0.919643,-0.161799
81,219,0.998477,0.918837,-0.071578
944,219,0.998660,0.947403,-0.017889


Log Predicted values that give R2 >0.9: 0


,DRUG_ID,r2_fitted,r2_predicted,r2_predicted_log


Results of direct fitting that gives R2 >0.9: 292


In [11]:
# for sigmoid function fitting showed that hyperparameters for all the coefficients are the same,
# and from the default values only gamma was different

short_test = TrainTestKernelByDrugLog(train, test, good_drug_ids, number_coefficients=4, kernel='sigmoid', 
                                   alpha=1, gamma=0.00001, degree=3, coef0=1)

# parameters: x0, L, k, b
# y = 1 / (L + np.exp(-k*(x-x0)))+b
# Wang 1.0 / (1.0 + np.exp((x-p)/s)   x - dosage [0, 1], p - position, s - shape parameter
short_test[['COSMIC_ID', 'DRUG_ID', "param_1","pred_param_1","param_2", "pred_param_2", 
            "param_3", "pred_param_3", "param_4", "pred_param_4"]].head()

fitting_cols = ["param_"+str(i) for i in range(1,5)]
pred_fitting_cols = ["pred_param_"+str(i) for i in range(1,5)]
log_pred_fitting_cols = ["log_pred_param_"+str(i) for i in range(1,5)]
fitting_function = "sigmoid_4_param"

short_test["r2_fitted"] = compute_r2_score(short_test, x_columns = conc_columns, y_columns = response_norm, 
                              fitting_parameters=fitting_cols, fitting_function = fitting_function)

short_test["r2_predicted"] = compute_r2_score(short_test, x_columns = conc_columns, y_columns = response_norm, 
                              fitting_parameters=pred_fitting_cols, fitting_function = fitting_function)

short_test["r2_predicted_log"] = compute_r2_score(short_test, x_columns = conc_columns, y_columns = response_norm, 
                              fitting_parameters=log_pred_fitting_cols, fitting_function = fitting_function)

# only one case when predicted values was a better fit to original data than directly fitted
print("Predicted values give higher R2 than fitted:")
display(short_test[short_test["r2_predicted"]>short_test["r2_fitted"]][["DRUG_ID","r2_fitted", "r2_predicted"]])

print("Predicted values that give R2 >0.9:", short_test[short_test["r2_predicted"]>0.9][["DRUG_ID","r2_fitted", "r2_predicted"]].shape[0])
display(short_test[short_test["r2_predicted"]>0.9][["DRUG_ID","r2_fitted", "r2_predicted", "r2_predicted_log"]].head())

print("Log Predicted values that give R2 >0.9:", short_test[short_test["r2_predicted_log"]>0.9].shape[0])
display(short_test[short_test["r2_predicted_log"]>0.9][["DRUG_ID","r2_fitted", "r2_predicted", "r2_predicted_log"]].head())

print("Results of direct fitting that gives R2 >0.9:", short_test[short_test["r2_fitted"]>0.9].shape[0])

Predicted values give higher R2 than fitted:


,DRUG_ID,r2_fitted,r2_predicted
1606,219,5.050434e-08,0.397983


Predicted values that give R2 >0.9: 56


,DRUG_ID,r2_fitted,r2_predicted,r2_predicted_log
134,170,0.998072,0.975915,-0.021794
1076,170,0.999686,0.986524,-0.013487
742,170,0.999448,0.982892,-0.031913
1909,170,0.997694,0.970117,-0.023219
1768,170,0.999126,0.913019,-0.053705


Log Predicted values that give R2 >0.9: 0


,DRUG_ID,r2_fitted,r2_predicted,r2_predicted_log


Results of direct fitting that gives R2 >0.9: 292


### Train and test with different kernels for different coefficients

In [13]:
best_parameters_kernels_123 = {
    "linear": {"alpha": {1: 200.0, 2: 100.0, 3: 100.0, 4: 50.0}},
               
    "sigmoid": {"gamma" :{1: 1e-05, 2: 1e-05, 3: 1e-05, 4: 0.01},
                "alpha":  {1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0},
                "coef0" : {1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0}
               },
               
    "rbf" : {"gamma": {1: 1e-05, 2: 1.0, 3: 1.0, 4: 0.001},
           "alpha" : {1: 10.0, 2: 10.0, 3: 10.0, 4: 5.0}
          }, 
               
    "polynomial" : { "gamma" : {1: 1e-05, 2: 1e-05, 3: 1e-05, 4: 1e-04},
                    "degree" : {1: 1.0, 2: 1.0, 3: 1.0, 4: 5.0}, 
                    "alpha" : {1: 5.0, 2: 1.0, 3: 1.0, 4: 0.01}
                   }}
               
best_kernels = {1: 'rbf', 2: 'rbf', 3: 'rbf', 4: 'rbf'}

In [17]:
short_test = TrainTestKernelByDrugLog(train, test, good_drug_ids, number_coefficients=4, 
                                   kernel_dict=best_kernels, kernel_param = best_parameters_kernels_123)

# parameters: x0, L, k, b
# y = 1 / (L + np.exp(-k*(x-x0)))+b
# Wang 1.0 / (1.0 + np.exp((x-p)/s)   x - dosage [0, 1], p - position, s - shape parameter
short_test[['COSMIC_ID', 'DRUG_ID', "param_1","pred_param_1","param_2", "pred_param_2", 
            "param_3", "pred_param_3", "param_4", "pred_param_4"]].head()

fitting_cols = ["param_"+str(i) for i in range(1,5)]
pred_fitting_cols = ["pred_param_"+str(i) for i in range(1,5)]
log_pred_fitting_cols = ["log_pred_param_"+str(i) for i in range(1,5)]
fitting_function = "sigmoid_4_param"

short_test["r2_fitted"] = compute_r2_score(short_test, x_columns = conc_columns, y_columns = response_norm, 
                              fitting_parameters=fitting_cols, fitting_function = fitting_function)

short_test["r2_predicted"] = compute_r2_score(short_test, x_columns = conc_columns, y_columns = response_norm, 
                              fitting_parameters=pred_fitting_cols, fitting_function = fitting_function)

short_test["r2_predicted_log"] = compute_r2_score(short_test, x_columns = conc_columns, y_columns = response_norm, 
                              fitting_parameters=log_pred_fitting_cols, fitting_function = fitting_function)

# only one case when predicted values was a better fit to original data than directly fitted
print("Predicted values give higher R2 than fitted:")
display(short_test[short_test["r2_predicted"]>short_test["r2_fitted"]][["DRUG_ID","r2_fitted", "r2_predicted"]])

print("Predicted values that give R2 >0.9:", short_test[short_test["r2_predicted"]>0.9][["DRUG_ID","r2_fitted", "r2_predicted"]].shape[0])
display(short_test[short_test["r2_predicted"]>0.9][["DRUG_ID","r2_fitted", "r2_predicted", "r2_predicted_log"]].head())

print("Log Predicted values that give R2 >0.9:", short_test[short_test["r2_predicted_log"]>0.9].shape[0])
display(short_test[short_test["r2_predicted_log"]>0.9][["DRUG_ID","r2_fitted", "r2_predicted", "r2_predicted_log"]].head())

print("Results of direct fitting that gives R2 >0.9:", short_test[short_test["r2_fitted"]>0.9].shape[0])

Predicted values give higher R2 than fitted:


,DRUG_ID,r2_fitted,r2_predicted
1606,219,5.050434e-08,0.529848


Predicted values that give R2 >0.9: 26


,DRUG_ID,r2_fitted,r2_predicted,r2_predicted_log
134,170,0.998072,0.930815,-0.028374
2547,170,0.999184,0.920415,-0.091051
742,170,0.999448,0.931055,-0.039714
1768,170,0.999126,0.960701,-0.059919
1254,170,0.999004,0.930898,-0.033480


Log Predicted values that give R2 >0.9: 0


,DRUG_ID,r2_fitted,r2_predicted,r2_predicted_log


Results of direct fitting that gives R2 >0.9: 292


**Conclusion:** Usage of log transformation of the parameters worsens the results!
It is better not to use log transfromations!